In [2]:
import numpy as np
import pdb
from PIL import Image
from scipy import ndimage
import os
import unittest
from scipy.ndimage import maximum_filter

In [94]:
png_file = Image.open("C:\\Data\\2.png")
array_image = np.array(png_file)
kernel_3d = np.random.rand(3, 3, 4)
kernel_3d_2 = np.random.rand(3, 3, 4) 
kernel_3d_3 = np.random.rand(3, 3, 4)
png_file_1 = Image.open("C:\\Data\\1.png")
array_image_1 = np.array(png_file_1)

In [52]:
kernel_3d

array([[[0.0229695 , 0.82028251, 0.26195054, 0.5230198 ],
        [0.71795052, 0.73789951, 0.51326136, 0.58539481],
        [0.66847521, 0.37797085, 0.17846777, 0.96771695]],

       [[0.2054206 , 0.07835007, 0.19237932, 0.25284656],
        [0.70348223, 0.90270746, 0.54695471, 0.70561571],
        [0.27347069, 0.48721745, 0.86218299, 0.45189815]],

       [[0.57164563, 0.91495283, 0.15168394, 0.37928622],
        [0.19260003, 0.84691538, 0.27741407, 0.94882001],
        [0.35470914, 0.77908422, 0.66216021, 0.93800158]]])

In [190]:
array_image.shape

(590, 586, 4)

In [198]:
def reshape_dimensions(input_height, input_width, filter_height, filter_width, stride):
    output_height = int(((input_height - filter_height) / stride) + 1)
    output_width  = int(((input_width - filter_width)  / stride) + 1)
    return (output_height, output_width)

In [234]:
reshape_dimensions(32, 32, 5, 5, 1)

(28, 28)

In [214]:
test_image = np.random.rand(4, 4, 4)

In [231]:
def forward_pass(image):
    shape = image.shape
    kernels = create_kernels(32)
    to_stack_values = []
    dimensions = reshape_dimensions(shape[0], shape[1], 3, 3, 1) 
    for i in range(len(kernels)):
        to_stack_values.append(np.squeeze(convolve_one_step(image, kernels[i]).reshape(dimensions[0], dimensions[1], 1)))
    stacked_filters = np.stack(to_stack_values, axis = 2)
    return stacked_filters.shape

In [233]:
forward_pass(array_image)

(588, 584, 32)

In [168]:
def create_kernels(number_to_create):
    every_kernel = []
    for f_u in range(number_to_create):
        every_kernel.append(np.random.rand(3, 3, 4))
    return every_kernel

In [174]:
jan = create_kernels(32)

In [211]:
len(jan)

32

In [32]:
def convolve_one_step(input_image, kernel_used):
    intermediate_values = []
    dimensions = input_image.shape
    final_values = []
    for i in range(dimensions[0] - 2):
        for j in range(dimensions[1] - 2):
            for k in range(dimensions[2]):
                slice_of_image = input_image[i : i + 3, j : j + 3, k]
                real_kernel = kernel_used[:, :, k]
                #pdb.set_trace()
                value = np.vdot(slice_of_image, real_kernel)
                intermediate_values.append(value)
                #pdb.set_trace()
                if len(intermediate_values) == 4:
                    final_values.append(sum_dimension(intermediate_values))
                    intermediate_values.clear()
    return np.array(final_values)

In [33]:
def sum_dimension(list_of_three):
    return sum(list_of_three)

In [203]:
image_after_conv_2 = np.array(convolve_one_step(array_image, kernel_3d)).reshape(dimensions[0], dimensions[1], 1)

In [204]:
image_after_conv_2.shape

(588, 584, 1)

In [41]:
image_after_conv_1 = np.array(convolve_one_step(array_image_1, kernel_3d)).reshape(341, 343, 1)

In [132]:
a = np.array(convolve_one_step(test_image, kernel_3d)).reshape(8, 8, 1)

In [133]:
b = np.array(convolve_one_step(test_image, kernel_3d_2)).reshape(8, 8, 1)

In [134]:
c = np.array(convolve_one_step(test_image, kernel_3d_3)).reshape(8, 8, 1)

In [135]:
a.shape

(8, 8, 1)

In [136]:
a = a.squeeze()

In [137]:
b = b.squeeze()

In [138]:
c = c.squeeze()

In [223]:
dupa_tymona = np.stack([a, b, c], axis = 2)

In [224]:
dupa_tymona.shape

(8, 8, 3)

In [14]:
def max_pooling(image_array):
    pooling_values = []
    for m in range(0, image_array.shape[0], 2):
        for n in range(0, image_array.shape[1], 2):
            for o in range(image_array.shape[2]):
                if n + 2 > image_array.shape[1]:
                    slice_of_pooling = image_array[m : m + 2, n - 1: n + 1 , o]
                else:
                    slice_of_pooling = image_array[m : m + 2, n : n + 2 , o]
                pooling_values.append(slice_of_pooling.max())
                #pdb.set_trace()
    return pooling_values

In [15]:
filtr = test_image[0 : 2, 0 : 2, 0]

In [16]:
after_maxpool = np.array(max_pooling(test_image)).reshape(5, 5, 4)

In [17]:
after_maxpool.shape

(5, 5, 4)

In [18]:
def ReLUv1(x):
    after_relu = []
    for r in x:
        if r > 0:
            after_relu.append(r)
        else:
            after_relu.append(0)
    return after_relu

In [19]:
def ReLUv2(x):
    return np.maximum(0, x)

In [20]:
last_pool = np.random.rand(2, 2, 1)

In [148]:
def fully_connected(after_last_max_pool, number_of_nodes):
    almost_values = []
    nodes_values = []
    flatten_array = after_last_max_pool.reshape(4)
    weights = np.random.rand(len(flatten_array), number_of_nodes)
    for x in range(weights.shape[1]):
        for y in range(weights.shape[0]):
            if y == weights.shape[0] - 1:
                almost_values.append(weights[y : y + 1, x] * flatten_array[y])
                nodes_values.append(sum(almost_values))
                almost_values.clear()
            else:
                almost_values.append(weights[y : y + 1, x] * flatten_array[y])
    return nodes_values

In [149]:
fully_connected(last_pool, 4)

[array([0.44608858]),
 array([0.35665557]),
 array([0.7646605]),
 array([0.45762526])]

In [157]:
lista = []
for i in range(10):
    kernel = np.random.rand(3, 3, 4)
    lista.append(kernel)

In [182]:
for i in range(32):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
